### 練習問題2.56
より多くの種類の式を扱えるようにこの基本的な微分プログラムを拡張するにはどうすればよいかを⽰せ。  
例えば、次の微分規則を実装せよ。  

$$
\frac{d(u^n)}{dx} = nu^{n−1} \frac{du}{dx}  
$$

実装にあたっては、derivプログラムに新しい節を追加し、また exponentiation?, base, exponent, make-exponentiationを適切 に定義せよ(冪乗の表現には**という記号を使ってもよい)。任意 の数の0乗は1であり、任意の数の1乗はそれ⾃⾝であるという 規則を組み込め。

In [1]:
(define (=number? exp num)(and (number? exp) (= exp num)))

; eは変数か?
(define (variable? x)(symbol? x)) 

; v1とv2は同じ変数か?
(define (same-variable? v1 v2)(and (variable? v1) (variable? v2) (eq? v1 v2)))

; eは和か?
(define (sum? x)(and (pair? x) (eq? (car x) '+)))

; 和eの加数
(define (addend s)(cadr s))

; 和eの被加数
(define (augend s)(caddr s))

; a1とa2の和を構築する
(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2)))
  )

;eは積か?
(define (product? x)(and (pair? x) (eq? (car x) '*))) 

; 積eの乗数
(define (multiplier p)(cadr p))

;積eの被乗数
(define (multiplicand p)(caddr p))

; m1とm2の積を構築する
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2)))
  )

;eはべき乗か?
(define (exponentiation? x)
  (and (pair? x) (eq? (car x) '**))) 

; べき乗の基数
(define (base x)(cadr x))

; 指数
(define (exponent x)(caddr x))

; べき数を構築する
(define (make-exponentiation m1 m2)
  (cond ((=number? m2 0) 1)
        ((=number? m2 1) m1)
        ;((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2) (> m2 0)) ((make-exponentiation (* m1 m1) (- m2 1))))
        (else (list '** m1 m2)))
  )


(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp) (make-sum (make-product (multiplier exp)
                                                (deriv (multiplicand exp) var))
                                  (make-product (deriv (multiplier exp) var) (multiplicand exp))))
        ((exponentiation? exp)
         (let ((u (base exp))
               (e (exponent exp)))
            (make-product (make-product e (make-exponentiation u (- e 1))) (deriv u var))
           )
         )
        (else (error "unknown expression type: DERIV" exp)))
  )

In [2]:
(deriv '(+ x 3) 'x)

1

In [3]:
(deriv '(* x y) 'x)

y

In [4]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [5]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))

In [6]:
(deriv '(** x 2) 'x)

(* 2 x)

In [7]:
(deriv '(** x 1) 'x)

1

In [8]:
(deriv '(** (+ x 1) 2) 'x)

(* 2 (+ x 1))